# Feed Forward neural network

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
train_dataset = dsets.MNIST(root = './root',
                            train = True , 
                            transform = transforms.ToTensor(),
                            download = True)

In [3]:
test_dataset = dsets.MNIST(root = './data',
                          train = False,
                          transform = transforms.ToTensor(),
                          download = True)

In [4]:
bs = 100
n_iters = 3000
num_epochs = n_iters/(len(train_dataset)/bs)
num_epochs = int(num_epochs)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = bs, 
                                          shuffle = True)

In [6]:
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = bs, 
                                          shuffle = False)

In [7]:
class FeedForwardNN(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(FeedForwardNN,self).__init__()
        self.fc1 = nn.Linear(input_dim,hidden_dim)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_dim,output_dim)
        
    def forward(self,x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        return out
    
        

In [9]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10
model = FeedForwardNN(input_dim, hidden_dim, output_dim)
model.cuda()

FeedForwardNN(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [10]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
for epoch in range(int(num_epochs)):
    
    for i,(images,labels) in enumerate(train_loader):
        
        images = Variable(images.view(-1,784).cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        
    correct = 0
    total = 0
    
    for images,labels in train_loader:
        
        images = Variable(images.view(-1,784).cuda())
        
        outputs = model(images.cuda())
        
        _, pred = torch.max(outputs.data,1)
        
        total += labels.size(0)
        
        correct += (pred.cpu() == labels.cpu()).sum()
        
    accuracy = 100*correct/total
    print(accuracy)

tensor(86)
tensor(89)
tensor(90)
tensor(91)
tensor(91)


In [13]:
correct = 0
total = 0
    
for images,labels in test_loader:
    
    images = Variable(images.view(-1,784).cuda())
    
    outputs = model(images)
        
    _, pred = torch.max(outputs.data,1)
        
    total += labels.size(0)
        
    correct += (pred.cpu() == labels.cpu()).sum()
        
accuracy = 100*correct/total
print(accuracy)

tensor(91)
